In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 16.50 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y1", label="ph")

/tmp/ipykernel_7119/4275043310.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-20 18:52:59.479504: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 18:53:01.660193: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 18:53:01.660284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 18:53:01.993658: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 18:53:02.709550: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 18:53:02.710843: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[25, 10], [30, 15], [45, 20]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[25, 10], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.9115604540002096
Stopped at epoch:  35
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.4597357410002587
Stopped at epoch:  13
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.4025476860001618
Stopped at epoch:  12
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.31003249300010793
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.2678486259997044
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.24537533100010478
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 7 ++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_1_6_0,0.801174,0.516314,-0.278992,0.547448,0.037196,0.058975,0.080958,0.069320
model_1_6_1,0.802473,0.524078,-0.271754,0.552126,0.036953,0.058028,0.080500,0.068603
model_1_6_4,0.802879,0.526194,-0.270637,0.553238,0.036877,0.057770,0.080429,0.068433
model_1_6_2,0.802968,0.525617,-0.265695,0.553953,0.036860,0.057841,0.080117,0.068324
model_1_6_3,0.802995,0.527364,-0.269924,0.553868,0.036855,0.057628,0.080384,0.068337
model_1_6_5,0.804180,0.534435,-0.262853,0.558221,0.036634,0.056766,0.079937,0.067670
model_1_6_6,0.804226,0.534750,-0.262626,0.558402,0.036625,0.056727,0.079922,0.067642
model_1_6_7,0.804398,0.536248,-0.262534,0.559050,0.036593,0.056545,0.079916,0.067543
model_1_6_8,0.804784,0.538812,-0.260410,0.560542,0.036520,0.056232,0.079782,0.067314


DataFrame salvo em ../results/metrics_1_6
+++++++++++ [1_7] | 1 ++++++++++++++++++
Elapsed time:  0.3069547690001855
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_7] | 2 ++++++++++++++++++
Elapsed time:  0.31739081900013844
Stopped at epoch:  7
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_7] | 3 ++++++++++++++++++
Elapsed time:  0.3630980550001368
Stopped at epoch:  8
1/1 [==============================] - 0s 26ms/step
+++++++++++ [1_7] | 4 ++++++++++++++++++
Elapsed time:  0.3421640629999274
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_7] | 5 ++++++++++++++++++
Elapsed time:  0.3791847220004456
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_7] | 6 ++++++++++++++++++
Elapsed time:  0.3429243089999545
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_7] | 7 ++++++++++++++++++
Elapsed time:  0.35471623400

2024-03-20 19:00:47.500978: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  1.2292279360003704
Stopped at epoch:  22
1/1 [==============================] - 0s 18ms/step
+++++++++++ [3_2] | 7 ++++++++++++++++++
Elapsed time:  0.47910448399989036
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [3_2] | 8 ++++++++++++++++++
Elapsed time:  0.7072700929998064
Stopped at epoch:  12
1/1 [==============================] - 0s 20ms/step
+++++++++++ [3_2] | 9 ++++++++++++++++++
Elapsed time:  0.5106384670002626
Stopped at epoch:  8
1/1 [==============================] - 0s 21ms/step
+++++++++++ [3_2] | 10 ++++++++++++++++++
Elapsed time:  0.4403091060003135
Stopped at epoch:  7
1/1 [==============================] - 0s 16ms/step
+++++++++++ [3_2] | 11 ++++++++++++++++++
Elapsed time:  0.7172179300000607
Stopped at epoch:  11
1/1 [==============================] - 0s 17ms/step
+++++++++++ [3_2] | 12 ++++++++++++++++++
Elapsed time:  0.4481655260001389
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/ste

...

1/1 [==============================] - 0s 22ms/step
+++++++++++ [12_3] | 21 ++++++++++++++++++
Elapsed time:  0.31682533799994417
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_3] | 22 ++++++++++++++++++
Elapsed time:  0.3341362869996374
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [12_3] | 23 ++++++++++++++++++
Elapsed time:  0.3349560759997985
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [12_3] | 24 ++++++++++++++++++
Elapsed time:  0.31314902099984465
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [12_3] | 25 ++++++++++++++++++
Elapsed time:  0.36755429599998024
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [12_4] | 1 ++++++++++++++++++
Elapsed time:  0.3939109680004549
Stopped at epoch:  9
1/1 [==============================] - 0s 16ms/step
+++++++++++ [12_4] | 2 ++++++++++++++++++
Elap

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_12_8_0,0.810392,0.621270,0.379382,0.546432,0.035471,0.073085,0.063736,0.068686
model_12_8_20,0.818275,0.504640,-0.255434,0.265155,0.033997,0.095591,0.128931,0.111280
model_12_8_19,0.819876,0.508433,-0.239594,0.272766,0.033697,0.094859,0.127304,0.110128
model_12_8_18,0.821562,0.512498,-0.222440,0.280986,0.033382,0.094075,0.125543,0.108883
model_12_8_1,0.822117,0.623454,0.334181,0.533481,0.033278,0.072663,0.068379,0.070647
model_12_8_17,0.823325,0.516853,-0.203940,0.289828,0.033052,0.093234,0.123643,0.107544
model_12_8_16,0.825157,0.521505,-0.183939,0.299350,0.032709,0.092337,0.121589,0.106102
model_12_8_15,0.827046,0.526539,-0.162481,0.309591,0.032356,0.091365,0.119385,0.104551
model_12_8_14,0.828987,0.531961,-0.139328,0.320642,0.031993,0.090319,0.117007,0.102878
model_12_8_2,0.830056,0.620835,0.287174,0.516712,0.031793,0.073168,0.073206,0.073186


DataFrame salvo em ../results/metrics_12_8
+++++++++++ [12_9] | 1 ++++++++++++++++++
Elapsed time:  0.3365954300006706
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_9] | 2 ++++++++++++++++++
Elapsed time:  0.19699376100015797
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_9] | 3 ++++++++++++++++++
Elapsed time:  0.298121933999937
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [12_9] | 4 ++++++++++++++++++
Elapsed time:  0.27123119499992754
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_9] | 5 ++++++++++++++++++
Elapsed time:  0.23332450699945184
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_9] | 6 ++++++++++++++++++
Elapsed time:  0.1893673429995033
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [12_9] | 7 ++++++++++++++++++
Elapsed time:  0.21

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_12_9_10,0.812686,0.084555,-0.401412,-0.011630,0.035042,0.139469,0.086553,0.114567
model_12_9_0,0.877692,0.867817,-0.024710,0.642883,0.022881,0.020138,0.063287,0.040443
model_12_9_9,0.879185,0.759035,-0.704535,0.390941,0.022602,0.036711,0.105274,0.068976
model_12_9_1,0.886429,0.869688,-0.073584,0.631670,0.021247,0.019853,0.066306,0.041713
model_12_9_8,0.891049,0.779045,-0.466568,0.466262,0.020382,0.033663,0.090577,0.060446
model_12_9_2,0.892878,0.866840,-0.113249,0.619464,0.020040,0.020287,0.068755,0.043096
model_12_9_7,0.897568,0.798218,-0.330007,0.514964,0.019163,0.030742,0.082143,0.054930
model_12_9_3,0.897582,0.859417,-0.141397,0.606954,0.019160,0.021418,0.070494,0.044512
model_12_9_4,0.900564,0.847850,-0.165870,0.592433,0.018602,0.023180,0.072005,0.046157
model_12_9_6,0.900833,0.816506,-0.248154,0.548995,0.018552,0.027955,0.077087,0.051076


DataFrame salvo em ../results/metrics_12_9
